In [1]:
from Networks.switchnorm import SwitchNormalization

In [2]:
import tensorflow
from tensorflow.python.client import device_lib
tensorflow.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
print("This program automatically segments Airways from input tiff images. ")
in1 = input("\nFour models were trained on Airways data. The average dice of these models are: - \n\n1. csUNet on RG = 0.94  \n2. cbUNet on RG = 0.93  \n3. cbUNet = 0.96 \n4. csUNet = 0.95 \n\nWhich model would you like to use? Enter 1/2/3/4 based on the order listed above. \nHowever, it is best to try all the models to see which one produces accurate segmentations.As of now cbUNet performs the best on new data.")
print(f"\nYou have chosen model {in1}")
tiff = input("\nEnter the path to tiff folder: ")
print("\nStarted Preprocessing...")


This program automatically segments Airways from input tiff images. 

Four models were trained on Airways data. The average dice of these models are: - 

1. csUNet on RG = 0.94  
2. cbUNet on RG = 0.93  
3. cbUNet = 0.96 
4. csUNet = 0.95 

Which model would you like to use? Enter 1/2/3/4 based on the order listed above. 
However, it is best to try all the models to see which one produces accurate segmentations.4
You have chosen model 4
Enter the path to tiff folder: C:\Users\UAB\Desktop\data
Started Preprocessing...


In [4]:
import os
import numpy as np
import SimpleITK as sitk

# Input and output directories
input_folder = tiff
nii_3d = '3d_nii'

# Get a list of TIFF files in the input folder
tif_files = [f for f in os.listdir(input_folder) if f.endswith('.tif')]

# Create the output folder if it doesn't exist
os.makedirs(nii_3d, exist_ok=True)

# Iterate through TIFF files and convert them to NIfTI
for tif_file in tif_files:
    tif_path = os.path.join(input_folder, tif_file)
    output_name = tif_file[:-3] + 'nii'  # Output NIfTI filename
    output_path = os.path.join(nii_3d, output_name)

    # Load TIFF image using SimpleITK
    tif_image = sitk.ReadImage(tif_path)

    # Convert SimpleITK image to NumPy array
    tif_array = sitk.GetArrayFromImage(tif_image)

    # Create a SimpleITK image from the NumPy array
    nii_image = sitk.GetImageFromArray(tif_array)

    # Save NIfTI image
    sitk.WriteImage(nii_image, output_path)

    #print(f"Converted {tif_file} to {output_name}")

#print("Conversion complete.")


In [5]:
import os
import glob
import nibabel as nib
from skimage.transform import resize

def slice_and_resize_nifti(nifti_path, output_dir):
    # Load the NIfTI image
    img = nib.load(nifti_path)
    data = img.get_fdata()
    #print(data.shape)

    # Iterate over each slice
    for slice_idx in range(data.shape[2]):
        # Extract the slice
        slice_data = data[:, :, slice_idx]

        # Resize the slice to 256x256
        resized_slice = resize(slice_data, (256,256), preserve_range=True)

        # Generate the output filename
        #filename = os.path.splitext(os.path.basename(nifti_path))[0]
        #print(filename)
        slice_filename = nifti_filename[:-6] + '_'+str(slice_idx)+ "_M.nii"
        #print(slice_filename)
        #print(slice_filename)
        output_path = os.path.join(output_dir, slice_filename)

        # Save the resized slice
        nib.save(nib.Nifti1Image(resized_slice, img.affine), output_path)
        #print(f"Saved {output_path}")
        with open(output_path, 'rb') as f:
            f.close()

# Example usage
input_dir =nii_3d
nii_2d = "2d_nii"
os.makedirs(nii_2d, exist_ok=True)

nifti_files = [file for file in os.listdir(input_dir) if file.endswith("M.nii")]

# Process each NIfTI file
for nifti_filename in nifti_files:
    nifti_path = os.path.join(input_dir, nifti_filename)
    slice_and_resize_nifti(nifti_path,nii_2d)

In [6]:
import os
import numpy as np
import tifffile as tiff
import nibabel as nib
from scipy.ndimage import rotate

# Set the paths to the NIfTI and TIFF files
input_folder = nii_2d

tiff_2d = "tiff_2d"
os.makedirs(tiff_2d, exist_ok=True)

# Get a list of all NIfTI files in the input folder
nifti_files = [file for file in os.listdir(input_folder) if file.endswith(".nii")]

for nifti_file in nifti_files:
    # Construct the paths for input and output files
    nifti_path = os.path.join(input_folder, nifti_file)
    tiff_file = nifti_file[:-4] + ".tif"
    tiff_path = os.path.join(tiff_2d, tiff_file)

    # Load the NIfTI file
    nifti_img = nib.load(nifti_path)
    nifti_data = nifti_img.get_fdata()
    #print(nifti_data.shape)
    #nifti_data = np.transpose(nifti_data, (2, 1, 0))
    #print(nifti_data.shape)#

    # Normalize the pixel values between 0 and 255
    nifti_data = (nifti_data - np.min(nifti_data)) / (np.max(nifti_data) - np.min(nifti_data))
    nifti_data = (nifti_data * 255).astype(np.uint8)


    # Save the rotated data as a TIFF image
    tiff.imsave(tiff_path, nifti_data)

    #print(f"Converted {nifti_path} to {tiff_path}")

#print("Conversion complete!")

(256, 256)
Converted 2d_nii\6S_ET556M_0_M.nii to tiff_2d\6S_ET556M_0_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_1000_M.nii to tiff_2d\6S_ET556M_1000_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_1001_M.nii to tiff_2d\6S_ET556M_1001_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_1002_M.nii to tiff_2d\6S_ET556M_1002_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_1003_M.nii to tiff_2d\6S_ET556M_1003_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_1004_M.nii to tiff_2d\6S_ET556M_1004_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_1005_M.nii to tiff_2d\6S_ET556M_1005_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_1006_M.nii to tiff_2d\6S_ET556M_1006_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_1007_M.nii to tiff_2d\6S_ET556M_1007_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_1008_M.nii to tiff_2d\6S_ET556M_1008_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_1009_M.nii to tiff_2d\6S_ET556M_1009_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_100_M.nii to tiff_2d\6S_ET556M_100_M.tif
(256, 256)
Converted 2d_nii\6S_ET556

(256, 256)
Converted 2d_nii\6S_ET556M_135_M.nii to tiff_2d\6S_ET556M_135_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_136_M.nii to tiff_2d\6S_ET556M_136_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_137_M.nii to tiff_2d\6S_ET556M_137_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_138_M.nii to tiff_2d\6S_ET556M_138_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_139_M.nii to tiff_2d\6S_ET556M_139_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_13_M.nii to tiff_2d\6S_ET556M_13_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_140_M.nii to tiff_2d\6S_ET556M_140_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_141_M.nii to tiff_2d\6S_ET556M_141_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_142_M.nii to tiff_2d\6S_ET556M_142_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_143_M.nii to tiff_2d\6S_ET556M_143_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_144_M.nii to tiff_2d\6S_ET556M_144_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_145_M.nii to tiff_2d\6S_ET556M_145_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_146_M.nii to tif

(256, 256)
Converted 2d_nii\6S_ET556M_29_M.nii to tiff_2d\6S_ET556M_29_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_2_M.nii to tiff_2d\6S_ET556M_2_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_300_M.nii to tiff_2d\6S_ET556M_300_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_301_M.nii to tiff_2d\6S_ET556M_301_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_302_M.nii to tiff_2d\6S_ET556M_302_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_303_M.nii to tiff_2d\6S_ET556M_303_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_304_M.nii to tiff_2d\6S_ET556M_304_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_305_M.nii to tiff_2d\6S_ET556M_305_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_306_M.nii to tiff_2d\6S_ET556M_306_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_307_M.nii to tiff_2d\6S_ET556M_307_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_308_M.nii to tiff_2d\6S_ET556M_308_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_309_M.nii to tiff_2d\6S_ET556M_309_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_30_M.nii to tiff_2d\

(256, 256)
Converted 2d_nii\6S_ET556M_440_M.nii to tiff_2d\6S_ET556M_440_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_441_M.nii to tiff_2d\6S_ET556M_441_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_442_M.nii to tiff_2d\6S_ET556M_442_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_443_M.nii to tiff_2d\6S_ET556M_443_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_444_M.nii to tiff_2d\6S_ET556M_444_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_445_M.nii to tiff_2d\6S_ET556M_445_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_446_M.nii to tiff_2d\6S_ET556M_446_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_447_M.nii to tiff_2d\6S_ET556M_447_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_448_M.nii to tiff_2d\6S_ET556M_448_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_449_M.nii to tiff_2d\6S_ET556M_449_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_44_M.nii to tiff_2d\6S_ET556M_44_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_450_M.nii to tiff_2d\6S_ET556M_450_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_451_M.nii to tif

(256, 256)
Converted 2d_nii\6S_ET556M_591_M.nii to tiff_2d\6S_ET556M_591_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_592_M.nii to tiff_2d\6S_ET556M_592_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_593_M.nii to tiff_2d\6S_ET556M_593_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_594_M.nii to tiff_2d\6S_ET556M_594_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_595_M.nii to tiff_2d\6S_ET556M_595_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_596_M.nii to tiff_2d\6S_ET556M_596_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_597_M.nii to tiff_2d\6S_ET556M_597_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_598_M.nii to tiff_2d\6S_ET556M_598_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_599_M.nii to tiff_2d\6S_ET556M_599_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_59_M.nii to tiff_2d\6S_ET556M_59_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_5_M.nii to tiff_2d\6S_ET556M_5_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_600_M.nii to tiff_2d\6S_ET556M_600_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_601_M.nii to tiff_2d

(256, 256)
Converted 2d_nii\6S_ET556M_741_M.nii to tiff_2d\6S_ET556M_741_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_742_M.nii to tiff_2d\6S_ET556M_742_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_743_M.nii to tiff_2d\6S_ET556M_743_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_744_M.nii to tiff_2d\6S_ET556M_744_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_745_M.nii to tiff_2d\6S_ET556M_745_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_746_M.nii to tiff_2d\6S_ET556M_746_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_747_M.nii to tiff_2d\6S_ET556M_747_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_748_M.nii to tiff_2d\6S_ET556M_748_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_749_M.nii to tiff_2d\6S_ET556M_749_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_74_M.nii to tiff_2d\6S_ET556M_74_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_750_M.nii to tiff_2d\6S_ET556M_750_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_751_M.nii to tiff_2d\6S_ET556M_751_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_752_M.nii to tif

Converted 2d_nii\6S_ET556M_896_M.nii to tiff_2d\6S_ET556M_896_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_897_M.nii to tiff_2d\6S_ET556M_897_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_898_M.nii to tiff_2d\6S_ET556M_898_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_899_M.nii to tiff_2d\6S_ET556M_899_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_89_M.nii to tiff_2d\6S_ET556M_89_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_8_M.nii to tiff_2d\6S_ET556M_8_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_900_M.nii to tiff_2d\6S_ET556M_900_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_901_M.nii to tiff_2d\6S_ET556M_901_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_902_M.nii to tiff_2d\6S_ET556M_902_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_903_M.nii to tiff_2d\6S_ET556M_903_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_904_M.nii to tiff_2d\6S_ET556M_904_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_905_M.nii to tiff_2d\6S_ET556M_905_M.tif
(256, 256)
Converted 2d_nii\6S_ET556M_906_M.nii to tiff_2d\6S_ET556M_

In [7]:
import numpy as np
import tifffile as tiff
import os

folder_path = nii_2d
input_npy = "input_npy_lungs"
os.makedirs(input_npy, exist_ok=True)
for filename in os.listdir(folder_path):
    if filename.endswith("M.nii"):
        # Load the 3D tiff image
        image_path = os.path.join(folder_path, filename)
        #image = tiff.imread(image_path)
        image = nib.load(image_path)
        image = image.get_fdata()
        #image = (image * 255).astype(np.uint8)
        #image = np.where(image>1,255,image)
        #image = np.where(image>1,1,image
        #print(image.shape)
        # Save the numpy array as a .npy file
        name = f"{filename[:-3]}" + 'npy' 
        np.save(os.path.join(input_npy, name), image)

In [8]:
print('Pre-processing completed')
print('Starting Prediction...')

Pre-processing completed
Starting Prediction...


In [9]:
import tensorflow as tf
#get the images
def gather_images(data_path):
    images = [f for f in os.listdir(data_path) if '_M.' in f]
    return images
#define custom objects
def Hausdorff_distance(y_true, y_pred, smooth=100):
        intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
        sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
        jac = (intersection + smooth) / (sum_ - intersection + smooth)
        return (1 - jac) * smooth

def dice_coef(y_true, y_pred):
    smooth = 1.0
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (
                K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(self, y_true, y_pred):
    loss = 1 - self._dice_coef(y_true, y_pred)
    return loss

class GELU(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(GELU, self).__init__(**kwargs)

    def call(self, inputs):
        return 0.5 * inputs * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (inputs + 0.044715 * tf.pow(inputs, 3))))

In [10]:
from keras.models import load_model
custom_objects = {
    'jaccard_distance': Hausdorff_distance,
    'dice_coef': dice_coef,
    'SwitchNormalization':SwitchNormalization
    # Add any other custom objects here as needed
}
if in1=='1':
    model = load_model(r"saved\unet_sn_mon_airways_train.h5", custom_objects=custom_objects)
if in1 == '2':
    model = load_model(r"saved\unet_bn_mon1_airways_train.h5", custom_objects=custom_objects)
if in1=='3':
    model = load_model(r"saved\UNet_airways_BN_kim_gt_train.h5", custom_objects=custom_objects)
if in1=='4':
    model = load_model(r"saved\SN_unet_airways_kim_gt_train.h5", custom_objects=custom_objects)

data_path = input_npy
images = gather_images(data_path) #get the input images using the defined functions
#print(len(images))
label_mapping = {name: index for index, name in enumerate(images)} #map the labels for prediction
labels = {name: label_mapping[name] for name in images}
model_name = 'test'

1136


In [11]:
#the below code does the prediction by passing the test images 
import numpy
import nibabel


label_list = []
predictions = []
pred_path1 = "npy_method_1"
pred_path2 = "t_method_1"
pred_path3 = "nii_method_1"
# Check if the directory exists
if not os.path.exists(pred_path1):
    # Create the directory
    os.makedirs(pred_path1)
    #print(f"Directory '{pred_path1}' created.")
if not os.path.exists(pred_path2):
    # Create the directory
    os.makedirs(pred_path2)
    #print(f"Directory '{pred_path2}' created.")
if not os.path.exists(pred_path3):
    # Create the directory
    os.makedirs(pred_path3)
    #print(f"Directory '{pred_path3}' created.")

for i, image in enumerate(images):
    image_stack = np.empty((1, 256,256, 1))  # create a tensor to hold images
    image = np.load(os.path.join(data_path, image))
    image_stack[0, ..., 0] = image[:,:]  #assign the image as a part of stack
    label_match = str(images[i][:-5] + 'M.npy') #match label
    label_list.append(label_match)

    img_prediction = model.predict(image_stack)  #model prediction

    for z, img_pred in enumerate(img_prediction): #saving the prediction as nii or npy
        img_numb = z + i
        image_save = img_pred[:, :]  #removing extra axis from the predicted image
        label = label_list[img_numb][:-5]
        filename = f"{label}{model_name}_P.npy"
        filename1 = f"{label}{model_name}_P.nii"
        np.save(os.path.join(pred_path1, filename), image_save)
       # the below code is optional if the above npy have to be saved as nii
        #image = np.load(os.path.join(pred_path1, filename))
        #affine = numpy.eye(4)
        #nifti_file = nibabel.Nifti1Image(image_save, affine)
        #nib.save(nifti_file, os.path.join(pred_path3, filename1))

In [12]:
def gather_set(data_path, phrase):
    path = data_path + '\\'
    set_of = [f for f in os.listdir(data_path) if phrase in f]
    return set_of

In [13]:
filepath_predictions = pred_path1
filepath_tensors = pred_path2
filepath_data = input_npy
images = gather_set(filepath_predictions, '.')
model_name = 'test'

In [14]:
id_list = []   

for i in range(len(images)):
    image_name = images[i]
    #last_underscore_index = image_name.rfind('_')
    #unique_id = image_name[:last_underscore_index + 1]
    unique_id = image_name[0:10]
    ##print(unique_id)
    id_list.append(unique_id)
    
unique_ids = list(set(id_list))
#print(len(unique_ids))

6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_
6S_ET556M_

In [15]:
import re
import os
import numpy as np

for i in range(len(unique_ids)):
    count = 0
    for filename in os.listdir(filepath_predictions):
        if filename.startswith(unique_ids[i]):
            count += 1

    #print(count)
    tensor = np.zeros((256,256, int(count)))

    for x in range(count):
        img_name = unique_ids[i] + str(x) + '_M.npy'
        img_path = os.path.join(filepath_data, img_name)

        try:
            image = np.load(img_path)
            img_slice = image
            tensor[:, :, x] = img_slice
        except FileNotFoundError:
            # Handle the case where the file is not found
            continue

    new_fname = unique_ids[i] + 'M.npy'
    np.save(os.path.join(filepath_tensors, new_fname), tensor)
    
import re
import os
import numpy as np

for i in range(len(unique_ids)):
    count = 0
    for filename in os.listdir(filepath_predictions):
        if filename.startswith(unique_ids[i]):
            count += 1

    #print(count)
    tensor = np.zeros((256,256, int(count)))

    for x in range(count):
        img_name = unique_ids[i] + str(x) + '_' + model_name + '_P.npy'
        img_path = os.path.join(filepath_predictions, img_name)

        try:
            image = np.load(img_path)
            img_slice = image[:, :, 1]
            tensor[:, :, x] = img_slice
        except FileNotFoundError:
            # Handle the case where the file is not found
            continue

    new_fname = unique_ids[i] + model_name + '_Prediction.npy'
    np.save(os.path.join(filepath_tensors, new_fname), tensor)


In [16]:
def gather_images1(data_path):
    images = []
    path = data_path + '\\'
    for f in os.listdir(data_path):
      if '_test' in f:
        images.append(f)
        
      else:
        continue
    images = np.array(images)

    indices = np.array(range(len(images))) # we will use this in the next step.

    return images

def gather_images2(data_path):
    images = []
    path = data_path + '\\'
    for f in os.listdir(data_path):
      if '_M.' in f:
        images.append(f)
        
      else:
        continue
    images = np.array(images)
    #segmentations = np.array(segmentations)

    indices = np.array(range(len(images))) # we will use this in the next step.

    return images

In [17]:
import nibabel as nib
images = gather_images1(filepath_tensors)
#print(len(images))
for i in range(len(images)):
    image = np.load(os.path.join(filepath_tensors, images[i]))
    affine = np.eye(4)
    nifti_file = nib.Nifti1Image(image, affine)
    filename = images[i][:-3]+'nii'
    nib.save(nifti_file, os.path.join(pred_path3, filename))
    
images = gather_images2(filepath_tensors)
#print(len(images))
for i in range(len(images)):
    image = np.load(os.path.join(filepath_tensors, images[i]))
    affine = np.eye(4)
    nifti_file = nib.Nifti1Image(image, affine)
    filename = images[i][:-3]+'nii'
    nib.save(nifti_file, os.path.join(pred_path3, filename))

1


In [ ]:
import shutil

# Specify the folder paths you want to remove
folder_paths = [nii_3d, nii_2d, pred_path1, pred_path2,tiff_2d, input_npy]  # Add more paths if needed

for folder_path in folder_paths:
    try:
        shutil.rmtree(folder_path)
        #print(f"Successfully removed the directory and its contents: {folder_path}")
    except Exception as e:
        continue




In [ ]:
print("Predictions completed. The processed input images and segmentations can be found in the folder 'nii_method_1'.")